In [1]:
%run ../scripts/notebook_settings.py
import sgkit as sg
import xarray as xr
import glob

First, determine samples to use and gender of the samples. Uses Vasili's cov files.

In [2]:
cov_summaries = glob.glob("/home/eriks/primatediversity/data/gVCFs_recalling_10_12_2024/Gorilla_gorilla_gorilla/cov/*")
df_l = []
for p in cov_summaries:
    df = pd.read_csv(p, sep="\t", names=["Name", "Contig", "start", "end", "length", "?", "coverage"])
    df["ID"] = p.split("/")[-1].split(".")[0]
    df_l.append(df)
df_cov = pd.concat(df_l)

In [3]:
df_cov["Contig_type"] = ["chrX" if x == "postPAR1" else "non-X" for x in df_cov.Name]
df_group = df_cov.groupby(["ID", "Contig_type"])["coverage"].median().reset_index()
non_X = df_group.loc[df_group.Contig_type == "non-X"]
df_cov["aut_cov"] = df_cov.ID.map(dict(zip(non_X.ID, non_X.coverage)))
df_cov["relative_cov"] = df_cov.coverage/df_cov.aut_cov

Use this df to create two sample cohorts, male and female.

In [4]:
df_x_only = df_cov.loc[df_cov.Contig_type == "chrX"]
df_x_only["Sex"] = [0 if x <= 0.75 else 1 for x in df_x_only.relative_cov]

/tmp/60261951/ipykernel_2615806/1432902879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_only["Sex"] = [0 if x <= 0.75 else 1 for x in df_x_only.relative_cov]


In [5]:
ds = sg.load_dataset("../data/Gorilla_gorilla_gorilla")
ds["sample_cohort"] = xr.DataArray(pd.Series(ds.sample_id.values).map(dict(zip(df_x_only.ID,df_x_only.Sex
                                                                              ))), dims="samples")

In [6]:
pd.Series(ds["variant_contig"]).unique() #unique or value_counts to determine contigs with data in them.

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23], dtype=int8)

In [34]:
# Step to handle my issues - remove empty contigs, that is contigs without any variants.
dstest = ds.sel(contigs=pd.Series(ds["variant_contig"]).unique())
IDs_to_keep = df_x_only.loc[df_x_only.aut_cov >= 20].ID
df_i_ID = pd.DataFrame({"ds_index": ds.samples, "ID": ds.sample_id})
dstest = dstest.sel(samples=df_i_ID.loc[df_i_ID.ID.isin(IDs_to_keep)].ds_index.values)

In [35]:
dstest

<xarray.Dataset> Size: 3GB
Dimensions:               (variants: 21463034, samples: 24, ploidy: 2,
                           contigs: 24, filters: 2, region_index_values: 21487,
                           region_index_fields: 6, alleles: 2)
Coordinates:
  * samples               (samples) int64 192B 0 3 4 6 7 8 ... 26 27 28 29 30 31
Dimensions without coordinates: variants, ploidy, contigs, filters,
                                region_index_values, region_index_fields,
                                alleles
Data variables: (12/17)
    call_genotype         (variants, samples, ploidy) int8 1GB dask.array<chunksize=(1000, 24, 2), meta=np.ndarray>
    call_genotype_mask    (variants, samples, ploidy) bool 1GB dask.array<chunksize=(1000, 24, 2), meta=np.ndarray>
    call_genotype_phased  (variants, samples) bool 515MB dask.array<chunksize=(1000, 24), meta=np.ndarray>
    contig_id             (contigs) object 192B dask.array<chunksize=(24,), meta=np.ndarray>
    contig_length         (contigs) float64 192B dask.array<chunksize=(24,), meta=np.ndarray>
    filter_id             (filters) object 16B dask.array<chunksize=(2,), meta=np.ndarray>
    ...                    ...
    variant_id            (variants) object 172MB dask.array<chunksize=(1000,), meta=np.ndarray>
    variant_id_mask       (variants) bool 21MB dask.array<chunksize=(1000,), meta=np.ndarray>
    variant_length        (variants) int16 43MB dask.array<chunksize=(1000,), meta=np.ndarray>
    variant_position      (variants) int32 86MB dask.array<chunksize=(1000,), meta=np.ndarray>
    variant_quality       (variants) float32 86MB dask.array<chunksize=(1000,), meta=np.ndarray>
    sample_cohort         (samples) int64 192B 1 1 0 1 1 1 1 1 ... 1 1 1 1 1 1 1
Attributes:
    source:            bio2zarr-0.1.5
    vcf_header:        ##fileformat=VCFv4.2\n##FILTER=<ID=PASS,Description="A...
    vcf_zarr_version:  0.2

In [45]:
ds_w = sg.window_by_position(dstest, size=10000)
ds_d = sg.diversity(ds_w) # It crashes on very large windows, but works in the 1k-10k range.

In [48]:
ds_d["stat_diversity"][1000:1200].values

array([[18.05555556, 20.50744488],
       [25.73737374, 34.11546643],
       [23.21616162, 25.85111001],
       [32.68852814, 36.82944493],
       [24.6010101 , 23.23387958],
       [17.42525253, 14.90468895],
       [14.38860029, 13.53286379],
       [26.47474747, 30.52507852],
       [19.37445887, 23.96713511],
       [12.92020202, 16.89312033],
       [13.36161616, 16.21941441],
       [16.45606061, 23.41589161],
       [28.94329004, 27.36440877],
       [ 9.30757576, 17.00904847],
       [ 3.77272727, 13.32464935],
       [ 8.65151515, 11.25577346],
       [18.17575758, 20.00812064],
       [24.68686869, 25.22419189],
       [18.97914863, 22.1714841 ],
       [18.22222222, 22.83110551],
       [ 5.09393939, 12.48653331],
       [ 8.02626263, 13.33789694],
       [12.25555556, 18.16096146],
       [ 6.18787879, 14.31113145],
       [ 5.78686869,  8.67933433],
       [ 4.38888889,  6.18088297],
       [ 6.8969697 ,  7.90843935],
       [12.44242424, 14.75114941],
       [18.45151515,